In [1]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/jobs?filter%5Bcategory%5D%5B0%5D=Technology&sort_by=title")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Search Open Nike Jobs








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description` if skills are not there then add accoring to you and for experience make it 1 years and make it dictonary type and not list remove some useless job and some location.
        In skills section add web development text MERN stack,typescript swift, kotlin etc randomly as well as ML tect stack just like dummy data remove other skills
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
    "jobs": {
        "CDN Security Engineer (WAF)": {
            "role": "CDN Security Engineer (WAF)",
            "experience": 1,
            "skills": "MERN stack, TypeScript, Swift, Kotlin, TensorFlow, Keras, Scikit-learn",
            "description": "Job Location: Atlanta, Georgia, United States"
        },
        "Director Software Engineering, IT": {
            "role": "Director Software Engineering, IT",
            "experience": 1,
            "skills": "Web Development, React, Node.js, Express, MongoDB, ML Stack: PyTorch, OpenCV",
            "description": "Job Location: Karnataka, Karnataka, India"
        },
        "Director, Insider Threat, IT": {
            "role": "Director, Insider Threat, IT",
            "experience": 1,
            "skills": "Web Development, Angular, JavaScript, HTML/CSS, ML Stack: Scikit-learn, NLTK",
            "description": "Job Location: Karnataka, Karnataka, India"
        },
        "Director, Software Engineering, SEC": {


In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'jobs': {'CDN Security Engineer (WAF)': {'role': 'CDN Security Engineer (WAF)',
   'experience': 1,
   'skills': 'MERN stack, TypeScript, Swift, Kotlin, TensorFlow, Keras, Scikit-learn',
   'description': 'Job Location: Atlanta, Georgia, United States'},
  'Director Software Engineering, IT': {'role': 'Director Software Engineering, IT',
   'experience': 1,
   'skills': 'Web Development, React, Node.js, Express, MongoDB, ML Stack: PyTorch, OpenCV',
   'description': 'Job Location: Karnataka, Karnataka, India'},
  'Director, Insider Threat, IT': {'role': 'Director, Insider Threat, IT',
   'experience': 1,
   'skills': 'Web Development, Angular, JavaScript, HTML/CSS, ML Stack: Scikit-learn, NLTK',
   'description': 'Job Location: Karnataka, Karnataka, India'},
  'Director, Software Engineering, SEC': {'role': 'Director, Software Engineering, SEC',
   'experience': 1,
   'skills': 'MERN stack, TypeScript, Kotlin, React Native, ML Stack: TensorFlow, Keras',
   'description': 'Job Location

In [8]:
type(json_res)

dict

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
links = collection.query(query_texts=["skills"], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [16]:
all_skills = [job_data["skills"] for job_data in json_res["jobs"].values()]
print(all_skills)

['MERN stack, TypeScript, Swift, Kotlin, TensorFlow, Keras, Scikit-learn', 'Web Development, React, Node.js, Express, MongoDB, ML Stack: PyTorch, OpenCV', 'Web Development, Angular, JavaScript, HTML/CSS, ML Stack: Scikit-learn, NLTK', 'MERN stack, TypeScript, Kotlin, React Native, ML Stack: TensorFlow, Keras', 'Web Development, Ruby on Rails, Python, JavaScript, ML Stack: PyTorch, Scikit-learn', 'MERN stack, TypeScript, Swift, Kotlin, ML Stack: TensorFlow, OpenCV', 'Web Development, React, Node.js, Express, MongoDB, ML Stack: PyTorch, Scikit-learn', 'Web Development, Angular, JavaScript, HTML/CSS, ML Stack: Scikit-learn, NLTK']


In [18]:
job

{'jobs': {'CDN Security Engineer (WAF)': {'role': 'CDN Security Engineer (WAF)',
   'experience': 1,
   'skills': 'MERN stack, TypeScript, Swift, Kotlin, TensorFlow, Keras, Scikit-learn',
   'description': 'Job Location: Atlanta, Georgia, United States'},
  'Director Software Engineering, IT': {'role': 'Director Software Engineering, IT',
   'experience': 1,
   'skills': 'Web Development, React, Node.js, Express, MongoDB, ML Stack: PyTorch, OpenCV',
   'description': 'Job Location: Karnataka, Karnataka, India'},
  'Director, Insider Threat, IT': {'role': 'Director, Insider Threat, IT',
   'experience': 1,
   'skills': 'Web Development, Angular, JavaScript, HTML/CSS, ML Stack: Scikit-learn, NLTK',
   'description': 'Job Location: Karnataka, Karnataka, India'},
  'Director, Software Engineering, SEC': {'role': 'Director, Software Engineering, SEC',
   'experience': 1,
   'skills': 'MERN stack, TypeScript, Kotlin, React Native, ML Stack: TensorFlow, Keras',
   'description': 'Job Location

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rakshit, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI & Software Solutions for Your Business Needs

Dear Hiring Manager,

I came across the job postings for various positions, including CDN Security Engineer (WAF), Director Software Engineering, IT, and others, and I believe AtliQ can be a valuable partner in fulfilling your technical requirements. As a business development executive at AtliQ, I'd like to introduce our company's capabilities in AI and software consulting.

At AtliQ, we specialize in providing tailored solutions to facilitate the seamless integration of business processes through automated tools. Our team of experts has empowered numerous enterprises with scalable, optimized, and cost-effective solutions, resulting in heightened overall efficiency. We have a strong portfolio in machine learning and software development, with expertise in MERN stack, TypeScript, Swift, Kotlin, TensorFlow, Keras, Scikit-learn, and other relevant technologies.

Our relevant portfolio includes:
- Machine Learning with Python